# <center> Serve the model using AWS Sagemaker </center>

In [ ]:
import sys
sys.path.append('../venv/lib/python3.7/site-packages')
import pandas as pd
import boto3
import json
import mlflow.sagemaker as mfs

Previously, we trained and log a machine learning model on our local tracking server. </br>
The goal in this part is to take your model and deploy it on AWS Sagemaker. Sagemaker will provides you an endpoint that you will call to get a prediction.

## <center> AWS Configuration </center>

You need to configure the aws command line. For this, connect to the aws console. Then go to My Security Credentials -> Access keys -> Create new Access Key -> Download keys file. It will download a csv with your access key id and secret access key.

![](Image/aws-configure.png)

Now that you have your keys, you can configure your aws command line. Open a terminal and launch the command aws configure. It will ask you to fill information like the keys you donwloaded, the region (paris is eu-west-3) and the output format (json)

![](Image/terminal.png)

Now let's create a role for deploy your model on sagemaker:

My Security Credentials -> Roles -> Create Role -> Select Sagemaker and click Next -> Click Next: Tags -> Next: Review -> Give a name to your role and click create role.

You will see that your role has been created.

![](Image/aws-configure2.png)

**Q1.** Deploy your model as an endpoint in AWS Sagemaker.

In [ ]:
run_id = "your_run_id" # the model you want to deploy - this run_id is available in the mlflow UI
region = "eu-west-3" # region of your account
aws_id = "your_id" # from the aws-cli output
arn = "arn:aws:iam::your_id:role/sagemaker" #arn of the role you created
app_name = "red-wine"
model_uri = "your_model_uri" % (run_id) # edit this path based on your working directory
image_url = aws_id + ".dkr.ecr." + region + ".amazonaws.com/mlflow-pyfunc:1.6.0" # change to your mlflow version

# Push an image to ecr with mlflow. Look at mlflow documentation and find which functions you need.
mfs.()

#Deploy your model. Same here
mfs.()

Your model should be deploy by now so you can now make an API call to get predictions.

**Q2.** Call your model for a prediction

In [ ]:
# Read the wine-quality csv file from the URL
csv_url = \
    'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
try:
    data = pd.read_csv(csv_url, sep=';')
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e)

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)

In [ ]:
def check_status(app_name):
    sage_client = boto3.client('sagemaker', region_name="eu-west-3")
    endpoint_description = sage_client.describe_endpoint(EndpointName=app_name)
    endpoint_status = endpoint_description["EndpointStatus"]
    return endpoint_status

def query_endpoint(app_name, input_json):
    client = boto3.session.Session().client("sagemaker-runtime", region)

    response = client.invoke_endpoint(
        EndpointName=app_name,
        Body=input_json,
        ContentType='application/json; format=pandas-split',
    )
    preds = response['Body'].read().decode("ascii")
    preds = json.loads(preds)
    print("Received response: {}".format(preds))
    return preds

## check endpoint status
print("Application status is: {}".format(check_status(app_name)))

## create test data and make inference from enpoint
query_input = "a_line_of_your_dataset".to_json(orient="split")
prediction1 = query_endpoint(app_name=app_name, input_json=query_input)